# 02 – Era Analysis

Explore pace, efficiency, and shooting trends across seasons.

## Setup

Ensures the project root is on `sys.path` when running from the notebooks directory.

In [ ]:
from pathlib import Path

import sys

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.pipeline import generate_team_season_summary


In [ ]:
from src.data_ingest import NBADataIngestor

ingestor = NBADataIngestor(prefer_sqlite=False)
summary_rs = generate_team_season_summary(
    ingestor,
    regular_season_only=True,
    save=True
)
summary_rs.head()

In [ ]:
summary_rs = generate_team_season_summary(
    ingestor=None,  # placeholder replaced below
)

## Pace Trend

Plot the mean pace by season using Matplotlib/Seaborn.

In [ ]:
pace_trend = summary_rs.groupby('SEASON_YEAR')['PACE'].mean().reset_index()
plt.figure(figsize=(10, 5))
sns.lineplot(data=pace_trend, x='SEASON_YEAR', y='PACE')
plt.title('Average Team Pace by Season (Regular Season)')
plt.xlabel('Season Year')
plt.ylabel('Pace (Possessions per 48)')
plt.grid(True, alpha=0.3)
plt.tight_layout()

## Three-Point Rate Trend

Visualize how shot profile has shifted over time.

In [ ]:
three_point_trend = summary_rs.groupby('SEASON_YEAR')['THREE_POINT_RATE'].mean().reset_index()
plt.figure(figsize=(10, 5))
sns.lineplot(data=three_point_trend, x='SEASON_YEAR', y='THREE_POINT_RATE')
plt.title('Average Team 3P Attempt Rate by Season')
plt.xlabel('Season Year')
plt.ylabel('3PA / FGA')
plt.grid(True, alpha=0.3)
plt.tight_layout()

## Offensive vs Defensive Efficiency

Scatter plot to inspect correlation and potential era clusters.

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=summary_rs, x='OFF_EFF_PER_100', y='DEF_EFF_PER_100', hue='SEASON_YEAR', palette='viridis', legend=False, alpha=0.6)
plt.title('Offensive vs Defensive Efficiency (Team-Season)')
plt.xlabel('Offensive Efficiency')
plt.ylabel('Defensive Efficiency')
plt.tight_layout()

## Next Ideas

- Segment into pre/post rule-change eras and compare distributions.
- Fit change-point detection on pace or 3P rate.
- Add playoff aggregates for comparison.